In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def scrape_historical_data(max_rows=1000):
    url = "https://coinmarketcap.com/currencies/bitcoin/historical-data/"
    driver = webdriver.Chrome()  # Hoặc webdriver.Firefox() nếu dùng Firefox
    driver.get(url)

    time.sleep(5)  # Cho 5s cho trang tai xong het

    historical_data = []
    total_rows = 0

    while total_rows < max_rows:
        # Lay cac hang duoc chi dinh
        rows = driver.find_elements(By.CSS_SELECTOR, ".cmc-table tbody tr")

        for row in rows[len(historical_data):]:  # Vong lap lay tung hang
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) == 7:  # Lay du 7 cot
                data = {
                    "date": cols[0].text,
                    "open": cols[1].text,
                    "high": cols[2].text,
                    "low": cols[3].text,
                    "close": cols[4].text,
                    "volume": cols[5].text,
                    "market_cap": cols[6].text
                }
                historical_data.append(data)

        total_rows = len(historical_data)

        print(f"Đã cào được {total_rows} hàng dữ liệu.")

        # So hang lon hon so hang can tim thi dung lai
        if total_rows >= max_rows:
            break

        # Cuon xuong
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text()='Load More']"))
            )
            # Tim nut Load
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
            time.sleep(2)  # Cho 2s
            load_more_button.click()
            time.sleep(5)  # Cho 5s
        except Exception as e:
            print("Không tìm thấy hoặc không thể nhấn nút Load More:", e)
            break

    driver.quit()
    return historical_data

# Luu csv
data = scrape_historical_data(max_rows=1000)
df = pd.DataFrame(data)
df.to_csv('bitcoin_historical_data.csv', index=False)

print("Dữ liệu đã được lưu thành công dưới dạng file CSV!")


Đã cào được 60 hàng dữ liệu.
Đã cào được 120 hàng dữ liệu.
Đã cào được 180 hàng dữ liệu.
Đã cào được 240 hàng dữ liệu.
Đã cào được 299 hàng dữ liệu.
Đã cào được 360 hàng dữ liệu.
Đã cào được 420 hàng dữ liệu.
Đã cào được 480 hàng dữ liệu.
Đã cào được 540 hàng dữ liệu.
Đã cào được 600 hàng dữ liệu.
Đã cào được 658 hàng dữ liệu.
Đã cào được 719 hàng dữ liệu.
Đã cào được 779 hàng dữ liệu.
Đã cào được 839 hàng dữ liệu.
Đã cào được 899 hàng dữ liệu.
Đã cào được 959 hàng dữ liệu.
Đã cào được 1017 hàng dữ liệu.
Dữ liệu đã được lưu thành công dưới dạng file CSV!


In [1]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tkinter import *
from tkinter import messagebox

In [2]:
df = pd.read_csv('bitcoin_historical_data.csv')


In [3]:
df.describe()

,date,open,high,low,close,volume,market_cap
count,1017,1017,1017,1017,1017,1017,1017
unique,1017,1017,1017,1016,1017,1017,1017
top,"Feb 24, 2022","$37,278.57","$38,968.84","$27,375.60","$38,332.61","$46,383,802,093","$727,072,550,320"
freq,1,1,1,2,1,1,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        1017 non-null   object
 1   open        1017 non-null   object
 2   high        1017 non-null   object
 3   low         1017 non-null   object
 4   close       1017 non-null   object
 5   volume      1017 non-null   object
 6   market_cap  1017 non-null   object
dtypes: object(7)
memory usage: 55.7+ KB


In [5]:
from sklearn.preprocessing import MinMaxScaler
# Chuyển cột 'date' về định dạng datetime
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y')
def preprocess_numeric_columns(df):
    object_columns = df.select_dtypes(include=['object']).columns


    for col in object_columns:
        try:
            # Chuẩn hóa và chuyển đổi về số
            df[col] = pd.to_numeric(
                df[col]
                .str.replace(',', '')  # Loại bỏ dấu phẩy
                .str.replace('$', '')  # Loại bỏ ký hiệu tiền tệ
                .str.replace(' ', '')  # Loại bỏ khoảng trắng
                .str.strip(),          # Loại bỏ khoảng trắng thừa
            )
        except Exception as e:
            print(f"Lỗi khi xử lý cột '{col}': {e}")

    return df
df = preprocess_numeric_columns(df)
# Hiển thị dữ liệu trước khi chuẩn hóa
print("Dữ liệu ban đầu:")
print(df.head())

# Lựa chọn các cột cần chuẩn hóa (giá trị số)
columns_to_scale = ['open', 'high', 'low', 'close', 'volume', 'market_cap']

# Sử dụng MinMaxScaler để chuẩn hóa
scaler = MinMaxScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# Hiển thị dữ liệu sau khi chuẩn hóa
print("\nDữ liệu sau khi chuẩn hóa:")
print(df.head())

Dữ liệu ban đầu:
        date      open       high       low     close        volume  \
0 2024-12-06  97074.22  102039.88  96514.88  99920.71   94534772658   
1 2024-12-05  98741.54  103900.47  91998.78  96593.57  149218945580   
2 2024-12-04  95988.53   99207.33  94660.52  98768.53   77199817112   
3 2024-12-03  95854.60   96297.20  93629.56  96002.16   67067810961   
4 2024-12-02  97276.01   98152.60  94482.86  95865.30   72680784305   

      market_cap  
0  1977630964721  
1  1911777565577  
2  1954156097012  
3  1899532461067  
4  1896994040043  

Dữ liệu sau khi chuẩn hóa:
        date      open      high       low     close    volume  market_cap
0 2024-12-06  0.976779  0.978772  0.991205  1.000000  0.619953    1.000000
1 2024-12-05  0.996813  1.000000  0.935883  0.960454  1.000000    0.960668
2 2024-12-04  0.963734  0.946454  0.968489  0.986305  0.499477    0.985979
3 2024-12-03  0.962125  0.913252  0.955860  0.953425  0.429061    0.953355
4 2024-12-02  0.979204  0.934421  0.966

In [6]:
Dt_train, Dt_test = train_test_split(df, test_size=0.3, shuffle=False)
X_train = np.array(Dt_train[['open','high','low','volume']].values) 
y_train = np.array(Dt_train['close'])
X_test = np.array(Dt_test[['open','high','low','volume']].values) 
y_test = np.array(Dt_test['close'])
print(X_train)

[[0.97677942 0.97877188 0.99120451 0.61995261]
 [0.99681344 1.         0.93588308 1.        ]
 [0.96373409 0.94645433 0.96848893 0.49947708]
 ...
 [0.00925233 0.00454902 0.01114281 0.06352913]
 [0.01122387 0.00587718 0.01100659 0.08113644]
 [0.01366173 0.00806413 0.01277681 0.07239953]]


In [7]:
# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Ridge Regression
ridge_model = Ridge(alpha=0.0001)
ridge_model.fit(X_train, y_train)

# Neural Network (MLPRegressor)
mlp_model = MLPRegressor(hidden_layer_sizes=(200,500),solver='lbfgs',max_iter=100,activation='identity' )
mlp_model.fit(X_train, y_train)

# Stacking
base_models = [('linear', linear_model), ('ridge', ridge_model), ('mlp', mlp_model)]
stacking_model = StackingRegressor(estimators=base_models)
stacking_model.fit(X_train, y_train)

StackingRegressor(estimators=[('linear', LinearRegression()),
                              ('ridge', Ridge(alpha=0.0001)),
                              ('mlp',
                               MLPRegressor(activation='identity',
                                            hidden_layer_sizes=(200, 500),
                                            max_iter=100, solver='lbfgs'))])

In [8]:
def evaluate_models(X_test, y_test):
    # Linear Regression
    linear_predictions = linear_model.predict(X_test)
    linear_r2 = r2_score(y_test, linear_predictions)
    linear_rmse = np.sqrt(mean_squared_error(y_test, linear_predictions))
    linear_mae = mean_absolute_error(y_test, linear_predictions)
    linear_nse = str((1-(np.sum((y_test-linear_predictions)**2)/np.sum((y_test-np.mean(y_test))**2))))

    # Ridge Regression
    ridge_predictions = ridge_model.predict(X_test)
    ridge_r2 = r2_score(y_test, ridge_predictions)
    ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_predictions))
    ridge_mae = mean_absolute_error(y_test, ridge_predictions)
    ridge_nse = str((1-(np.sum((y_test-ridge_predictions)**2)/np.sum((y_test-np.mean(y_test))**2))))

    # MLPRegressor
    mlp_predictions = mlp_model.predict(X_test)
    mlp_r2 = r2_score(y_test, mlp_predictions)
    mlp_rmse = np.sqrt(mean_squared_error(y_test, mlp_predictions))
    mlp_mae = mean_absolute_error(y_test, mlp_predictions)
    mlp_nse = str((1-(np.sum((y_test-mlp_predictions)**2)/np.sum((y_test-np.mean(y_test))**2))))

    # StackingRegressor
    stacking_predictions = stacking_model.predict(X_test)
    stacking_r2 = r2_score(y_test, stacking_predictions)
    stacking_rmse = np.sqrt(mean_squared_error(y_test, stacking_predictions))
    stacking_mae = mean_absolute_error(y_test, stacking_predictions)
    stacking_nse = str((1-(np.sum((y_test-stacking_predictions)**2)/np.sum((y_test-np.mean(y_test))**2))))

    return {
        'Linear Regression': {'R2': linear_r2, 'NSE': linear_nse, 'RMSE': linear_rmse, 'MAE': linear_mae},
        'Ridge Regression': {'R2': ridge_r2, 'NSE': ridge_nse, 'RMSE': ridge_rmse, 'MAE': ridge_mae},
        'MLPRegressor': {'R2': mlp_r2, 'NSE': mlp_nse, 'RMSE': mlp_rmse, 'MAE': mlp_mae},
        'StackingRegressor': {'R2': stacking_r2, 'NSE': stacking_nse, 'RMSE': stacking_rmse, 'MAE': stacking_mae}
    }

def predict_new_data_linear():
    try:
        new_data = [float(entry1.get()), float(entry2.get()), float(entry3.get()), float(entry4.get())]
        metrics = evaluate_models(X_test, y_test)
        result_text = f"Giá trị dự đoán: {linear_model.predict([new_data])[0]}\n"
        for model_name, scores in metrics.items():
            result_text += f"{model_name} - R2: {scores['R2']}, NSE: {scores['NSE']}, RMSE: {scores['RMSE']}, MAE: {scores['MAE']}\n"

        result_label.config(text=result_text)
    except ValueError:
        messagebox.showerror("Lỗi", "Đầu vào không hợp lệ. Vui lòng nhập đúng dữ liệu số.")

def predict_new_data_ridge():
    try:
        new_data = [float(entry1.get()), float(entry2.get()), float(entry3.get()), float(entry4.get())]
        metrics = evaluate_models(X_test, y_test)
        result_text = f"Giá trị dự đoán: {ridge_model.predict([new_data])[0]}\n"
        result_label.config(text=result_text)
        for model_name, scores in metrics.items():
            result_text += f"{model_name} - R2: {scores['R2']}, NSE: {scores['NSE']}, RMSE: {scores['RMSE']}, MAE: {scores['MAE']}\n"

        result_label.config(text=result_text)
    except ValueError:
        messagebox.showerror("Lỗi", "Đầu vào không hợp lệ. Vui lòng nhập đúng dữ liệu số.")

def predict_new_data_mlp():
    try:
        new_data = [float(entry1.get()), float(entry2.get()), float(entry3.get()), float(entry4.get())]
        metrics = evaluate_models(X_test, y_test)
        result_text = f"Giá trị dự đoán: {mlp_model.predict([new_data])[0]}\n"
        result_label.config(text=result_text)
        for model_name, scores in metrics.items():
            result_text += f"{model_name} - R2: {scores['R2']}, NSE: {scores['NSE']}, RMSE: {scores['RMSE']}, MAE: {scores['MAE']}\n"
        result_label.config(text=result_text)
    except ValueError:
        messagebox.showerror("Lỗi", "Đầu vào không hợp lệ. Vui lòng nhập đúng dữ liệu số.")

def predict_new_data_stacking():
    try:
        new_data = [float(entry1.get()), float(entry2.get()), float(entry3.get()), float(entry4.get())]
        metrics = evaluate_models(X_test, y_test)
        result_text = f"Giá trị dự đoán: {stacking_model.predict([new_data])[0]}\n"
        result_label.config(text=result_text)
        for model_name, scores in metrics.items():
            result_text += f"{model_name} - R2: {scores['R2']}, NSE: {scores['NSE']}, RMSE: {scores['RMSE']}, MAE: {scores['MAE']}\n"
        result_label.config(text=result_text)
    except ValueError:
        messagebox.showerror("Lỗi", "Đầu vào không hợp lệ. Vui lòng nhập đúng dữ liệu số.")

In [9]:
# Create main window
window = Tk()
window.title("Form Dự Đoán Giá Crypto")
window.geometry("800x400")

# Create and pack widgets
header_label = Label(window, text="Nhập dữ liệu để dự đoán giá crypto:")
header_label.pack()
label1 = Label(window, text="Open:")
label1.pack()
entry1 = Entry(window)
entry1.pack()

label2 = Label(window, text="High:")
label2.pack()
entry2 = Entry(window)
entry2.pack()

label3 = Label(window, text="Low:")
label3.pack()
entry3 = Entry(window)
entry3.pack()

label4 = Label(window, text="Volume:")
label4.pack()
entry4 = Entry(window)
entry4.pack()

predict_button = Button(window, text="Giá trị dự đoán dựa trên mô hình LinearRegression:", command=predict_new_data_linear)
predict_button.pack()

predict_button = Button(window, text="Giá trị dự đoán dựa trên mô hình RidgeRegression:", command=predict_new_data_ridge)
predict_button.pack()

predict_button = Button(window, text="Giá trị dự đoán dựa trên mô hình MLPRegression:", command=predict_new_data_mlp)
predict_button.pack()

predict_button = Button(window, text="Giá trị dự đoán dựa trên mô hình StackingRegression:", command=predict_new_data_stacking)
predict_button.pack()

result_label = Label(window, text="")
result_label.pack()

# Run the main loop
window.mainloop()
